# Dependencies

In [94]:
import numpy as np
import tensorflow as tf
import sympy as sp
from sympy.solvers.solveset import linsolve

# Activation Functions

In [95]:
def leaky_relu(x,alpha=0.2):
    if x>0:
        return x
    else:
        return alpha*x
    
def ilr_case1(y):
    return y

def ilr_case2(y,alpha=0.2):
    return y/alpha

def identity(x):
    return x
def minus_identity(x):
    return -x

# Preimages functions

In [96]:
def linear_reverse(y):
    return y

def relu_reverse(y):
    if type(y)==sp.core.add.Add or type(y)==sp.core.symbol.Symbol:
        return sp.Function("iR")(y)
    if y>0:
        return y
    else:
        return sp.Symbol('R-')
    
def leaky_relu_reverse(y,alpha=0.2):
    if type(y)==sp.core.add.Add or type(y)==sp.core.symbol.Symbol:
        return sp.Function("iLR")(y)
    if y>0:
        return y
    else:
        return y*(1/alpha)
    
def tanh_reverse(y):
    return sp.atanh(y)
    

def sigmoid_reverse(y):
    if type(y)==sp.core.add.Add or type(y)==sp.core.symbol.Symbol:
        return sp.Function("logit")(y)
    return sp.log(y/(1-y))

# Neural Network Preimage computation

In [97]:
def convert2bin(n: int, length: int) -> list:
    res=[0]*length
    i=1
    while n!=0:
        res[-i]=n%2
        n=n//2
        i+=1
    return res
    
def set_cases(container : list,funs : list) -> list:
    """
    Return the 2^n inequalities to verify where n = length(container)
    """
    res=[]
    n=len(container)
    N=2**n
    for i in range(N):
        template=convert2bin(i,n)
        temp=container.copy()
        for k in range(n):
            if template[k]:
                temp[k]=funs[0](temp[k])
            else:
                temp[k]=funs[1](temp[k])
        res.append(temp)
    return res
            

def process_system(system : sp.matrices.dense.MutableDenseMatrix) -> tuple:
    """
    Solve system
    """
    ncols=system.cols
    target=system[:ncols-1,:]
    constraints=system[ncols-1:,:]
    target=linsolve(target)
    mat_temp=constraints[:,:ncols-1]*sp.Matrix(make_target(target))-constraints[:,ncols-1:]
    sol_temp=sp.solve(mat_temp)
    target=make_target(target)
    for i in range(len(target)):
        target[i]=target[i].subs(sol_temp)
    return target,sol_temp


def get_i_weights(weights : list, i : int) -> list:
    """
    Return values of ith columns of the weights matrix
    """
    res=[]
    for w in weights:
        res.append(w[i])
    return res

def reverse_layer(layer : tf.keras.layers.Layer ,table : dict) -> list:
    """
    Invert the layer "layer" according to constraints given by table (dict)
    """
    target_output=table["sols"]
    sets=table["sets"]
    temp=layer.get_weights()
    weights=temp[0]
    bias=temp[1]
    fa_name=layer.activation.__name__
    temp_table_list = process_inverse(fa_name,target_output,sets)
    new_table_list= []
    for table in temp_table_list:
        list_eq=[]
        target_output=table["sols"]
        sets=table["sets"]
        for i in range(len(target_output)):
            extra=target_output[i]-bias[i]
            list_eq.append(get_i_weights(weights,i)+[extra])

        system=sp.Matrix(list_eq)
        if system.rows>=system.cols:
            if len(system.free_symbols)>system.rows-system.cols:
                targ,targ_sol=process_system(system)
                for symb in targ_sol:
                    sets[symb]["equal"]=targ_sol[symb]
                new_table_list.append({"sols":targ,"sets":sets})
            else:
                new_table_list.append({"sols":process_system_eps(system),"sets":sets})
        else:
            new_table_list.append({"sols":make_target(linsolve(system)),"sets":sets})
    return new_table_list


def make_target(finiteset : sp.sets.sets.FiniteSet) -> list:
    """
    Convert finiteset in list
    """
    res=[]
    for obj in finiteset.args[0]:
        res.append(obj)
    return res

def reverse_network(model : tf.keras.Model) -> dict:
    """
    Returns the preimage of the Tensorflow model
    """
    n=model.layers[-1].output.shape[1]
    target_output=[]
    for i in range(n):
        target_output.append(sp.Symbol("y"+str(i)))
    list_layers=model.layers
    table={"sols":target_output,"sets":{}}
    table_list = []
    return reverse_layers(list_layers,table,len(list_layers)-1,table_list)

def reverse_layers(list_layers : list,table : dict ,i : int ,table_list : list)-> dict :
    """
    Iteratively inverts layers in list_layers
    """
    new_table_list= []
    temp_table=reverse_layer(list_layers[i],table)
    
    new_table_list += temp_table
    
    if i==0:
        table_list += new_table_list
    else:
        for new_table in new_table_list:
            reverse_layers(list_layers,new_table,i-1,table_list)
        if i==len(list_layers)-1:
            return table_list
        
def standard_sets(symbols : list ) -> dict:
    res={}
    for symbol in symbols:
        res[symbol]={"lower_than":sp.oo, "greater_than":-sp.oo}
    return res
        
def process_inverse(fa_name : str ,target_output : list ,sets : dict) -> list:
    """
    Choose the correct method to invert the layer according to the activation
    """
    table_list = []
    symbols=get_free_symb(target_output,["tau","y"])
    fa_inverse=fa_name+'_reverse'
    if fa_inverse=='linear_reverse':
        sets=standard_sets(symbols)
        table_list.append({"sols":target_output,"sets":sets})
    if fa_inverse=='leaky_relu_reverse':
        container=target_output
        cases=set_cases(container,[identity,minus_identity])
        temp_target=set_cases(container,[ilr_case1,ilr_case2])
        for i in range(len(cases)):
            new_sets=solve_linear_inequalities(cases[i],symbols)
            table_list.append({"sols":temp_target[i],"sets":new_sets})
    return table_list

def char_part(symb : sp.core.symbol.Symbol) -> str:
    """
    Return the name of the symbol "symb" without it index
    """
    res=''
    for char in str(symb):
        if char.isnumeric():
            return res
        else:
            res+=char
    return res

def get_free_symb(eqs : list,names : list) -> set:
    """
    Return free variables of the equation "eq" of name in "name"
    """
    res=set()
    for eq in eqs:
        symb_dict=eq.free_symbols
        for symb in symb_dict:
            if char_part(symb) in names:
                res.add(symb)
    return res

# Solving linear inequalities systems

In [98]:
        
def _find_pivot(inequalities : list , vars_ : set) -> sp.core.symbol.Symbol:
    """
    Returns a variable that has at least two coefficients with opposite
    sign in a system of inequalities.

    Examples
    ========

    >>> eq1 = 2*x - 3*y + z + 1
    >>> eq2 = x - y + 2*z - 2
    >>> eq3 = x + y + 3*z + 4
    >>> eq4 = x - z

    >>> inequalities = [eq1, eq2, eq3, eq4]
    >>> vars={x,y,z}
    >>> _find_pivot(inequalities,symbols)
    y
    """
    memory = {}
    for eq in inequalities:
        symbols=vars_.intersection(eq.free_symbols)
        for symbol in symbols:
            if not (symbol in memory.keys()):
                memory[symbol] = [False, False]
            coeff = eq.coeff(symbol)
            if coeff > 0:
                memory[symbol][0] = True
            else:
                memory[symbol][1] = True
            if memory[symbol] == [True, True]:
                return symbol


def _split_min_max(inequalities : list, pivot : sp.core.symbol.Symbol) -> tuple:
    """
    Returns expressions that are less than or greater than the pivot
    (have a coefficient on the pivot that is negative or positive).
    Inequalities that do not contain a pivot are returned as a list.

    Examples
    ========

    >>> eq1 = 2*x - 3*y + z + 1
    >>> eq2 = x - y + 2*z - 2
    >>> eq3 = x + y + 3*z + 4
    >>> eq4 = x - z

    >>> inequalities = [eq1, eq2, eq3, eq4]
    >>> pivot = y
    >>> _split_min_max(inequalities, pivot)
    (Min(2*x/3 + z/3 + 1/3, x + 2*z - 2), -x - 3*z - 4, [x - z])
    """

    greater_than = []
    lower_than = []
    extra = []
    for eq in inequalities:
        coeff = eq.coeff(pivot)
        if coeff > 0:
            greater_than.append(-(eq - (pivot * coeff)) / coeff)
        elif coeff < 0:
            lower_than.append(-(eq - (pivot * coeff)) / coeff)
        else:
            extra.append(eq)
    return sp.Min(*lower_than), sp.Max(*greater_than), extra


def _merge_mins_maxs(mins, maxs,symbols : set) -> list:
    """
    Build the system of inequalities which verify that all equations
    of maxs are greater than those of mins.

    Examples
    ========
    
    >>> maxs = -x - 3*z - 4
    >>> mins = Min((2*x + z + 1)/3, x + 2*z - 2)
    >>> symbols = {x,z}
    >>> _merge_mins_maxs(mins, maxs,symbols)
    [2*x + 5*z + 2, 5*x/3 + 10*z/3 + 13/3]
    """
    if not isinstance(mins, sp.Min):
        mins = [mins]
    else:
        mins = mins.args

    if not isinstance(maxs, sp.Max):
        maxs = [maxs]
    else:
        maxs = maxs.args
    return [i - j for i in mins for j in maxs]


def _fourier_motzkin(inequalities: list, symbols: set) -> tuple:
    """
    Eliminate variables of system of linear inequalities by using
    Fourier-Motzkin elimination algorithm

    Examples
    ========

    >>> eq1 = 2*x - 3*y + z + 1
    >>> eq2 = x - y + 2*z - 2
    >>> eq3 = x + y + 3*z + 4
    >>> eq4 = x - z
    >>> symbols = {x,y,z}

    >>> ie, d = _fourier_motzkin([eq1, eq2, eq3, eq4],symbols)
    >>> ie
    [3*x/2 + 13/10, 7*x/5 + 2/5]
    >>> assert set(d) == set([y, z])
    >>> d[y]
    (Min(2*x/3 + z/3 + 1/3, x + 2*z - 2) > y, y > -x - 3*z - 4)
    >>> d[z]
    (x > z, z > Max(-x/2 - 13/10, -2*x/5 - 2/5))
    """
    pivot = _find_pivot(inequalities,symbols)
    res = {}
    while pivot != None:
        mins, maxs, extra = _split_min_max(inequalities, pivot)
        res[pivot] = {"lower_than":mins, "greater_than":maxs}
        inequalities = _merge_mins_maxs(mins, maxs,symbols) + extra
        pivot = _find_pivot(inequalities,symbols)
    return inequalities, res


def _pick_var(inequalities : list ,vars_ : set) -> set:
    """
    Return a free variable of the system of inequalities

    Examples
    ========

    >>> eq1 = 2*x - 3*y + z + 1
    >>> eq2 = x - y + 2*z - 2
    >>> eq3 = x + y + 3*z + 4
    >>> eq4 = x - z
    >>> vars={x,y,z}

    >>> inequalities = [eq1, eq2, eq3, eq4]
    >>> _pick_var(inequalities,vars_)
    x
    """
    symbols = vars_
    for eq in inequalities:  # should already be in canonical order
        symbols=symbols.intersection(eq.free_symbols)
    return symbols
        #for symb in symbols:
            #return symb


def _fourier_motzkin_extension(inequalities  : list ,symbols : set) -> dict:
    """
    Extension of the Fourier-Motzkin algorithm to the case where
    inequalities do not contain variables that have at least two
    coefficients with opposite sign.

    Examples
    ========

    >>> eq1 = 2*x - 3*y + z + 1
    >>> eq2 = x - y + 2*z - 2
    >>> eq3 = x - y + 3*z + 4
    >>> eq4 = x + z
    >>> symbols = {x,y,z}

    >>> d = _fourier_motzkin_extension([eq1, eq2, eq3, eq4],symbols)
    >>> assert set(d) == {x}
    >>> d[x]
    (oo > x, x > Max(-z, y - 3*z - 4, y - 2*z + 2, 3*y/2 - z/2 - 1/2))
    >>> _fourier_motzkin_extension([x - 3, 5 - x])
    {x: (5 > x, x > 3)}
    """

    res = {}
    pivot = next(iter(_pick_var(inequalities,symbols)))
    while pivot and inequalities:
        mins, maxs, extra = _split_min_max(inequalities, pivot)
        res[pivot] = {"lower_than":mins, "greater_than":maxs}
        inequalities = extra
        pivot = next(iter(_pick_var(inequalities,symbols)))
    return res




def solve_linear_inequalities(eqs : list ,symbols : set) -> dict:
    """
    Solve a system of linear inequalities

    Parameters
    ==========

    inequalities: list of sympy equations
        The system of inequalities to solve. All equations in the list
        are assumed to be linear and greater than 0. The system must
        be expressed as follows:

        2x - 3y +  z + 1 > 0
        x  -  y + 2z - 2 > 0
        x  +  y + 3z + 4 > 0
        x       -  z     > 0

    Examples
    ========

    >>> eq1 = 2*x - 3*y + z + 1
    >>> eq2 = x - y + 2*z - 2
    >>> eq3 = x + y + 3*z + 4
    >>> eq4 = x - z

    >>> symbols = {x,y,z}

    >>> d = solve_linear_inequalities([eq1, eq2, eq3, eq4],symbols)
    >>> assert set(d) == set([x, y, z])
    >>> d[x]
    (oo > x, x > -2/7)
    >>> d[y]
    (Min(x + 1/3, 3*x - 2) > y, y > -4*x - 4)
    >>> d[z]
    (x > z, z > Max(-2*x + 3*y - 1, -x/2 + y/2 + 1, -x/3 - y/3 - 4/3))

    Explanation
    ===========

    x = 2 is valid because: oo > 2 > -2/7
    y = -1 is valid because: Min(x + 1/3, 3*x - 2) > -1 > -4*x - 4
    z = 1 is valid because: x > 1 > Max(-2*x + 3*y - 1, -x/2 + y/2 + 1, -x/3 - y/3 - 4/3)
    """
    eqs, res1 = _fourier_motzkin(eqs,symbols)
    res2 = _fourier_motzkin_extension(eqs,symbols)
    return {**res1, **res2}    

# Tests

In [99]:
from sympy.abc import x,y,z
    
def find_pivot_test():
    eq1 = 2*x - 3*y + z + 1
    eq2 = x - y + 2*z - 2
    eq3 = x + y + 3*z + 4
    eq4 = x - z
    inequalities = [eq1, eq2, eq3, eq4]
    symbols={x,y,z}
    assert _find_pivot(inequalities,symbols) == y, "Wrong Symbol : should be y"
    
def split_min_max_test():
    eq1 = 2*x - 3*y + z + 1
    eq2 = x - y + 2*z - 2
    eq3 = x + y + 3*z + 4
    eq4 = x - z
    inequalities = [eq1, eq2, eq3, eq4]
    pivot = y
    lower,upper,extra = _split_min_max(inequalities, pivot)
    assert lower == sp.Min(2*x/3 + z/3 + sp.Rational('1/3'), x + 2*z - 2), "Wrong result : Min(2*x/3 + z/3 + 1/3, x + 2*z - 2)"
    assert upper == -x - 3*z - 4,"Wrong result : should be -x - 3*z - 4"
    assert extra == [x-z], "Wrong result : should be [x-z]"
    
def merge_mins_maxs_test():
    maxs = -x - 3*z - 4
    mins = sp.Min((2*x + z + 1)/3, x + 2*z - 2)
    symbols = {x,z}
    min_max = _merge_mins_maxs(mins, maxs,symbols)
    assert min_max == [2*x + 5*z + 2, 5*x/3 + 10*z/3 + sp.Rational('13/3')],"Wrong result : should be [2*x + 5*z + 2, 5*x/3 + 10*z/3 + 13/3]"
    
def fourier_motzkin_test():
    eq1 = 2*x - 3*y + z + 1
    eq2 = x - y + 2*z - 2
    eq3 = x + y + 3*z + 4
    eq4 = x - z
    inequalities = [eq1, eq2, eq3, eq4]
    symbols = {x,y,z}
    ie, d = _fourier_motzkin(inequalities,symbols)
    assert ie == [3*x/2 + sp.Rational('13/10'), 7*x/5 + sp.Rational('2/5')],"Wrong result : should be [3*x/2 + 13/10, 7*x/5 + 2/5]"
    assert d == {y: {'lower_than': sp.Min(2*x/3 + z/3 + sp.Rational('1/3'), x + 2*z - 2),'greater_than': -x - 3*z - 4},z: {'lower_than': x, 'greater_than': sp.Max(-x/2 - sp.Rational('13/10'), -2*x/5 - sp.Rational('2/5'))}},"Wrong Result"

def pick_var_test():
    eq1 = 2*x - 3*y + z + 1
    eq2 = x - y + 2*z - 2
    eq3 = x + y + 3*z + 4
    eq4 = x - z
    inequalities = [eq1, eq2, eq3, eq4]
    symbols = {x,y,z}   
    var = _pick_var(inequalities,symbols)
    assert var == {x, z} ,"Wrong symbols : should be {x,z}"
    
def fourier_motzkin_extension_test():
    eq1 = 2*x - 3*y + z + 1
    eq2 = x - y + 2*z - 2
    eq3 = x + y + 3*z + 4
    eq4 = x - z
    inequalities = [eq1, eq2, eq3, eq4]
    symbols = {x,y,z}
    d = _fourier_motzkin_extension(inequalities,symbols)
    assert d == {z: {'lower_than': x,'greater_than': sp.Max(-2*x + 3*y - 1, -x/2 + y/2 + 1, -x/3 - y/3 - sp.Rational('4/3'))}},"Wrong result"

    
def solve_linear_inequalities_test():
    eq1 = 2*x - 3*y + z + 1
    eq2 = x - y + 2*z - 2
    eq3 = x + y + 3*z + 4
    eq4 = x - z
    inequalities = [eq1, eq2, eq3, eq4]
    symbols = {x,y,z}       
    d = solve_linear_inequalities(inequalities,symbols)
    assert d == {y: {'lower_than': sp.Min(2*x/3 + z/3 + sp.Rational('1/3'), x + 2*z - 2),'greater_than': -x - 3*z - 4},z: {'lower_than': x, 'greater_than': sp.Max(-x/2 - sp.Rational('13/10'), -2*x/5 - sp.Rational('2/5'))},x: {'lower_than': sp.oo, 'greater_than': sp.Rational('-2/7')}}, "Wrong Result"

def reverse_network_layers_test():
    initializer1 = tf.keras.initializers.RandomNormal(mean=0.0, stddev=10, seed=1)
    initializer2 = tf.keras.initializers.RandomNormal(mean=0.0, stddev=10, seed=2)
    model_test=tf.keras.Sequential()
    model_test.add(tf.keras.Input(shape=(1,)))
    model_test.add(tf.keras.layers.Dense(2,activation=tf.nn.leaky_relu,kernel_initializer = initializer1))
    model_test.add(tf.keras.layers.Dense(1,activation="linear",kernel_initializer = initializer2))
    obj = reverse_network(model_test)
    assert abs(float((sp.simplify(obj[0]["sols"][0] - 4.45910974145516*sp.Symbol("y0"))).coeff(sp.Symbol("y0")))) < 1e-8, "Wrong Result"
    

In [100]:
find_pivot_test()
split_min_max_test()
merge_mins_maxs_test()
fourier_motzkin_test()
pick_var_test()
fourier_motzkin_extension_test()
solve_linear_inequalities_test()
reverse_network_layers_test()

In [101]:
initializer1 = tf.keras.initializers.RandomNormal(mean=0.0, stddev=10, seed=1)
initializer2 = tf.keras.initializers.RandomNormal(mean=0.0, stddev=10, seed=2)
model_test=tf.keras.Sequential()
model_test.add(tf.keras.Input(shape=(1,)))
model_test.add(tf.keras.layers.Dense(2,activation=tf.nn.leaky_relu,kernel_initializer = initializer1))
model_test.add(tf.keras.layers.Dense(1,activation="linear",kernel_initializer = initializer2))
model_test.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 2)                 4         
                                                                 
 dense_45 (Dense)            (None, 1)                 3         
                                                                 
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________


In [102]:
res=reverse_network(model_test)
res

[{'sols': [4.45910974145516*y0],
  'sets': {tau0: {'lower_than': 0,
    'greater_than': -0.534743*y0,
    'equal': -4.39683004032092*y0},
   y0: {'lower_than': oo, 'greater_than': 0}}},
 {'sols': [0.1315787052128*y0],
  'sets': {tau0: {'lower_than': oo,
    'greater_than': Max(0, -0.534743*y0),
    'equal': -0.648704828194444*y0}}},
 {'sols': [-0.159886056251948*y0],
  'sets': {tau0: {'lower_than': Min(0, -0.534743*y0),
    'greater_than': -oo,
    'equal': 0.157652892353731*y0}}},
 {'sols': [0.891818571465864*y0],
  'sets': {tau0: {'lower_than': -0.534743*y0,
    'greater_than': 0,
    'equal': -4.39681496461072*y0},
   y0: {'lower_than': 0, 'greater_than': -oo}}}]

In [103]:
y0=-1
x0=0.1315787052128*y0
print(model_test(np.asarray([[x0]])))

tf.Tensor([[-0.9999999]], shape=(1, 1), dtype=float32)


# Demo

In [104]:
initializer1 = tf.keras.initializers.RandomNormal(mean=0.0, stddev=10, seed=3)
initializer2 = tf.keras.initializers.RandomNormal(mean=0.0, stddev=10, seed=4)
model=tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(2,activation=tf.nn.leaky_relu,kernel_initializer = initializer1))
model.add(tf.keras.layers.Dense(1,activation="linear",kernel_initializer = initializer2))
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 2)                 4         
                                                                 
 dense_47 (Dense)            (None, 1)                 3         
                                                                 
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________


In [105]:
res=reverse_network(model)
res

[{'sols': [0.0271569943940365*y0],
  'sets': {tau0: {'lower_than': Min(0, 0.115559*y0),
    'greater_than': -oo,
    'equal': 0.0910329813787625*y0}}},
 {'sols': [0.00654220028895761*y0],
  'sets': {tau0: {'lower_than': 0.115559*y0,
    'greater_than': 0,
    'equal': 0.109650572600109*y0},
   y0: {'lower_than': oo, 'greater_than': 0}}},
 {'sols': [0.0146877825771227*y0],
  'sets': {tau0: {'lower_than': 0,
    'greater_than': 0.115559*y0,
    'equal': 0.049234950609625*y0},
   y0: {'lower_than': 0, 'greater_than': -oo}}},
 {'sols': [0.00543140397060871*y0],
  'sets': {tau0: {'lower_than': oo,
    'greater_than': Max(0, 0.115559*y0),
    'equal': 0.0910329605025787*y0}}}]

Example:

Input:

res = reverse_network(model)

res

Output:

    [{'sols': [0.0271569943940365*y0],
      'sets': {tau0: {'lower_than': Min(0, 0.115559*y0),
        'greater_than': -oo,
        'equal': 0.0910329813787625*y0}}},
     {'sols': [0.00654220028895761*y0],
      'sets': {tau0: {'lower_than': 0.115559*y0,
        'greater_than': 0,
        'equal': 0.109650572600109*y0},
       y0: {'lower_than': oo, 'greater_than': 0}}},
     {'sols': [0.0146877825771227*y0],
      'sets': {tau0: {'lower_than': 0,
        'greater_than': 0.115559*y0,
        'equal': 0.049234950609625*y0},
       y0: {'lower_than': 0, 'greater_than': -oo}}},
     {'sols': [0.00543140397060871*y0],
      'sets': {tau0: {'lower_than': oo,
        'greater_than': Max(0, 0.115559*y0),
        'equal': 0.0910329605025787*y0}}}]
   
As you can see, there are a total of 4 solutions given for the preimage as expected.
'sols' are the solutions, an expression depending on the y (here only y0 because of the unique output neuron)
'sets' are the additional constraints. Here tau0 is the only one. For the second solution, it
has to be lower than 0.115559*y0 and greater than 0.

In this particular case, tau0 is equal to 0.109650572600109*y0.

If we are looking for a solution that gives 1 as an output, this second solution fits all the criterions.

In [106]:
y0=1
x0=0.00654220028895761*y0
print(model(np.asarray([[x0]])))

tf.Tensor([[0.99999994]], shape=(1, 1), dtype=float32)



Among the 4 given solutions above, only one would be valid according to the chosen y0.
In this example, y0 would be equal to 1.
The solutions have constraints given in "sets" : 

    {'sols': [0.00654220028895761*y0],
     'sets': {tau0: {'lower_than': 0.115559*y0,
     'greater_than': 0,
     'equal': 0.109650572600109*y0}
    
tau0 is the constraint and is equal to 0.109650572600109*y0 = 0.109650572600109 in this example since y0=1.
tau0 must be lower than 0.115559*y0 = 0.115559 so this condition is valid. It also must be greater than 0. This second condition is valid, so the right solution must be given by:

    'sols': [0.00654220028895761*y0]

    
Let's try this:

y0=1
x0=0.00654220028895761*y0
print(model(np.asarray([[x0]])))

Output: tf.Tensor([[0.99999994]], shape=(1, 1), dtype=float32)

It works!

You will see that with y0=1, other constraints will be False.

Now you can try yourself with other random neural networks.